<a href="https://colab.research.google.com/github/newgirlcarol/TFB103/blob/main/04_%E6%9D%8E%E6%98%80%E8%AB%AD(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 1. Original code >> Accuracy = 95.75%
> 3 層 Conv2D (16, 32, 64) w/ size=3x3, padding = same \
> epochs 12 tims \
> Optimizer = SGD
#### 2. First change >> Accuracy = 10.32%
> 2 層 Conv2D (32, 64) w/ seze=3x3, padding = valid \
> epochs 10 tims \
> Optimizer = Adam
#### 3. Second change >> Accuracy = 97.69%
> 4 層 Conv2D (16, 32, 64, 128) w/ size=5x5, padding = same \
> epochs 10 tims \
> Optimizer = SGD(lr=0.2) \
 
    小結: 多增加一層Conv. 且kernal size 改為5x5 有提高準確率; 只用兩層且一開始設32個filter並將優化器改用Adam,準確率顯著較低。


In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten # CNN ConV transfrom to Dense layer need flatten
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

### 1. 讀入 MNSIT 數據集

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有 **灰階**, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [5]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [6]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [8]:
# Sample from Tr. Yenlung
model = Sequential()
model.add(Conv2D(16, (3,3), padding='same', # 16 個 3x3 大小的filter
                input_shape=(28,28,1),
                activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), padding='same', # 一樣大小的filter，但filter個數增加2的倍數
                activation='relu'))

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(54, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [9]:
# Practice_1 移調16個的filter & padding = 'Valid' 
model_valid = Sequential()
model_valid.add(Conv2D(32, (3,3), padding='valid', 
                input_shape=(28,28,1),
                activation='relu'))

model_valid.add(MaxPooling2D(pool_size=(2,2)))
model_valid.add(Conv2D(64, (3,3), padding='valid', 
                activation='relu'))

model_valid.add(MaxPooling2D(pool_size=(2,2)))

model_valid.add(Flatten())
model_valid.add(Dense(54, activation='relu'))
model_valid.add(Dense(10, activation='softmax'))

In [10]:
# Practice_2 改變add one more filter & padding = 'same' 
model_dense = Sequential()
model_dense.add(Conv2D(16, (5,5), padding='same',
                input_shape=(28,28,1),
                activation='relu'))
model_dense.add(MaxPooling2D(pool_size=(2,2)))

model_dense.add(Conv2D(32, (5,5), padding='same',
                activation='relu'))
model_dense.add(MaxPooling2D(pool_size=(2,2)))

model_dense.add(Conv2D(64, (5,5), padding='same', 
                activation='relu'))
model_dense.add(MaxPooling2D(pool_size=(2,2)))

model_dense.add(Conv2D(128, (5,5), padding='same', 
                activation='relu'))
model_dense.add(MaxPooling2D(pool_size=(2,2)))

model_dense.add(Flatten())
model_dense.add(Dense(54, activation='relu'))
model_dense.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [11]:
model.summary()
print()
model_valid.summary()
print()
model_dense.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

#### 組裝

In [12]:
# model_valid w/ Adam
model.compile(loss='mse', optimizer=SGD(lr=0.087),
             metrics=['accuracy'])

model_valid.compile(loss='mse', optimizer=Adam(lr=0.087),
             metrics=['accuracy'])

model_dense.compile(loss='mse', optimizer=SGD(lr=0.2),
             metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


### 4. step 2: fit

In [13]:
# I change epoch w/ 10 times
model.fit(x_train, y_train, batch_size=128, epochs=12)

model_valid.fit(x_train, y_train, batch_size=128, epochs=10)

model_dense.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/12
469/469 [==============================] - 10s 9ms/step - loss: 0.0898 - accuracy: 0.1040
Epoch 2/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0894 - accuracy: 0.1924
Epoch 3/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0887 - accuracy: 0.3388
Epoch 4/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0858 - accuracy: 0.4906
Epoch 5/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0541 - accuracy: 0.6552
Epoch 6/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0231 - accuracy: 0.8535
Epoch 7/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0152 - accuracy: 0.9029
Epoch 8/12
469/469 [==============================] - 4s 8ms/step - loss: 0.0119 - accuracy: 0.9241
Epoch 9/12
469/469 [==============================] - 4s 9ms/step - loss: 0.0100 - accuracy: 0.9352
Epoch 10/12
469/469 [==============================] - 4s 9ms/step - loss: 0.0087 - accuracy: 0.944

### Step 3. 預測

In [14]:
predict_x=model.predict(x_test) 
result = np.argmax(predict_x, axis=1)

predict_new=model_valid.predict(x_test) 
result = np.argmax(predict_new, axis=1)

predict_dense=model_dense.predict(x_test) 
result = np.argmax(predict_dense, axis=1)

### 看看測試資料表現如何

In [15]:
# Original
loss, acc = model.evaluate(x_test, y_test)
# change padding=valid and filter 
loss_new, acc_new = model_valid.evaluate(x_test, y_test)
# 
loss_new, acc_dense = model_dense.evaluate(x_test, y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.0035 - accuracy: 0.9769


In [16]:
print(f'老師版的資料的正確率為 {acc*100:.2f}%')
print(f'學生P1版的資料的正確率為 {acc_new*100:.2f}%')
print(f'學生P2版的資料的正確率為 {acc_dense*100:.2f}%')

老師版的資料的正確率為 95.75%
學生P1版的資料的正確率為 10.32%
學生P2版的資料的正確率為 97.69%


In [17]:
# def my_predict(n):
#     print('我可愛的 CNN 預測是', result[n])
#     X = x_test[n].reshape(28,28)
#     plt.imshow(X, cmap='Greys')

In [18]:
# from ipywidgets import interact_manual

In [19]:
# interact_manual(my_predict, n=(0, 9999));

### 把我們的 model 存起來

In [20]:
# model.save('myCNNmodel.h5')z